# Analyse Data

In [2]:
from pathlib import Path
import park_and_ride_analysis

module_dir = Path(park_and_ride_analysis.__file__).parent
project_dir = module_dir.parent.parent
data_dir = project_dir / "data"

processed_data_dir = data_dir / "processed"

output_dir = project_dir / "output"
output_dir.mkdir(parents=True, exist_ok=True)

In [3]:
import pandas as pd
import ast

df = pd.read_csv(
    processed_data_dir / "einpendelverflechtungen_darmstadt_2022.csv", dtype={
        "Wohnort (Code)": str,
        "coordinates": object
    }
)
df["coordinates"] = df["coordinates"].apply(lambda x: None if pd.isna(x) else  ast.literal_eval(x))

df_sorted = df.sort_values(by='Anteil (Prozent)', ascending=False)
df_sorted.head(20)

,Wohnort (Code),Wohnort,Gesamt (Anzahl),Männlich (Anzahl),Weiblich (Anzahl),Anteil (Prozent),Luftlinienentfernung (km),coordinates
1553,064320008008,"Griesheim, Stadt",5116.0,2457.0,2659.0,5.6,8.0,"(49.8615899, 8.5762043)"
1521,064120000000,"Frankfurt am Main, Stadt",5033.0,2799.0,2234.0,5.5,26.3,"(50.1106444, 8.6820917)"
1568,064320023023,"Weiterstadt, Stadt",4068.0,1975.0,2093.0,4.5,6.4,"(49.9178, 8.5924)"
1563,064320018018,"Pfungstadt, Stadt",3867.0,1892.0,1975.0,4.3,10.6,"(49.7943, 8.5877)"
1559,064320014014,Mühltal,2473.0,1144.0,1329.0,2.7,8.4,"(49.8077, 8.6941)"
1561,064320016016,"Ober-Ramstadt, Stadt",2411.0,1144.0,1267.0,2.7,9.4,"(49.8152, 8.7548)"
1564,064320019019,"Reinheim, Stadt",2263.0,1157.0,1106.0,2.5,12.9,"(49.8364879, 8.8238238)"
1565,064320020020,Roßdorf,2288.0,1131.0,1157.0,2.5,7.1,"(50.7026664, 10.2163793)"
1567,064320022022,Seeheim-Jugenheim,2176.0,1055.0,1121.0,2.4,13.7,"(49.7598, 8.6632)"
1579,064330011011,"Riedstadt, Büchnerstadt",2092.0,1083.0,1009.0,2.3,15.0,"(49.8425212, 8.4829447)"


In [4]:
import folium

darmstadt_coords = (49.8728, 8.6512)
map = folium.Map(location=darmstadt_coords, zoom_start=10)

filtered_df = df[(df['coordinates'].notnull()) & (df['Luftlinienentfernung (km)'] < 50)]
for _, row in filtered_df.iterrows():
    folium.CircleMarker(
        location=row['coordinates'], radius=row['Gesamt (Anzahl)'] / 1000, color='blue', fill=True, fill_color='blue',
        fill_opacity=0.6, popup=f"{row['Wohnort']}: {row['Gesamt (Anzahl)']} commuters"
    ).add_to(map)

map.save(output_dir / 'commuter_map.html')


In [5]:
import osmnx as ox

graph = ox.load_graphml(filepath=processed_data_dir / "darmstadt_drive_50km.graphml")

In [16]:
# import folium
# import osmnx as ox
# import networkx as nx
# from tqdm import tqdm
# import branca.colormap as cm
# from collections import defaultdict

# # Dictionary to store the cumulative intensity for each road segment
# segment_intensity = defaultdict(int)

# # Function to add routes to the dictionary with cumulative intensity
# def accumulate_route_intensity(graph, origin_point, destination_point, intensity):
#     origin_node = ox.distance.nearest_nodes(graph, origin_point[1], origin_point[0])
#     destination_node = ox.distance.nearest_nodes(graph, destination_point[1], destination_point[0])
#     route = nx.shortest_path(graph, origin_node, destination_node, weight='length')

#     for i in range(len(route) - 1):
#         segment = (route[i], route[i + 1])
#         segment_intensity[segment] += intensity


# # Accumulate intensity for each route
# for _, row in tqdm(filtered_df.iterrows(), total=filtered_df.shape[0]):
#     try:
#         accumulate_route_intensity(
#             graph, origin_point=row['coordinates'], destination_point=darmstadt_coords, intensity=row['Gesamt (Anzahl)']
#         )
#     except Exception as e:
#         print(f"Error processing {row}: {e}")


# # Create a custom green-to-red color map
# colormap = cm.LinearColormap(
#     colors=['green', 'lime', 'yellow', 'orange', 'red', 'darkred'], vmin=min(segment_intensity.values()),
#     vmax=max(segment_intensity.values()), caption='Commuter Intensity'
# )
# map.add_child(colormap)

# def add_segments_to_map(map, graph, segment_intensity):
#     for segment, intensity in segment_intensity.items():
#         coords = [(graph.nodes[segment[0]]['y'], graph.nodes[segment[0]]['x']),
#                   (graph.nodes[segment[1]]['y'], graph.nodes[segment[1]]['x'])]
#         color = colormap(intensity)
#         popup = folium.Popup(f'Commuters: {intensity}', parse_html=True)
#         folium.PolyLine(coords, color=color, weight=5, opacity=0.8, popup=popup).add_to(map)

# add_segments_to_map(map, graph, segment_intensity)

# map.save(output_dir / 'commuter_map_with_intensity.html')

In [17]:
import folium
import osmnx as ox
import networkx as nx
from tqdm import tqdm
import branca.colormap as cm
from collections import defaultdict
import pandas as pd

darmstadt_coords = (49.8728, 8.6512)
map = folium.Map(location=darmstadt_coords, zoom_start=10)

# Dictionary to store the cumulative intensity for each road segment
segment_intensity = defaultdict(int)

# Define multiple destination points in Darmstadt
darmstadt_points = [
    (49.870000, 8.628380),  # Darmstadt HBF / Rhein-Neckar
    (49.891864, 8.653267),  # Darmstadt Nord
    (49.888186, 8.632706),  # Darmstadt Otto-Röhm / Carl-Schenk B3
    (49.853369, 8.646571),  # Eschollbrücker Straße
    (49.874747, 8.673300),  # Darmstadt Ost
    (49.857348, 8.668419),  # Merck Stadion
    (49.879264, 8.670220),  # Spessart Ring
    (49.8594138, 8.6348138), # Darmstadt West / Riedstraße
]


# Function to find the nearest point from a list of points
def find_nearest_point(graph, point, points_list):
    nearest_point = None
    shortest_distance = float('inf')
    for p in points_list:
        distance = ox.distance.euclidean(point[0], point[1], p[0], p[1])
        if distance < shortest_distance:
            nearest_point = p
            shortest_distance = distance
    return nearest_point


# Function to add routes to the dictionary with cumulative intensity
def accumulate_route_intensity(graph, origin_point, destination_point, intensity):
    origin_node = ox.distance.nearest_nodes(graph, origin_point[1], origin_point[0])
    destination_node = ox.distance.nearest_nodes(graph, destination_point[1], destination_point[0])
    route = nx.shortest_path(graph, origin_node, destination_node, weight='length')

    for i in range(len(route) - 1):
        segment = (route[i], route[i + 1])
        segment_intensity[segment] += intensity

# Accumulate intensity for each route
for _, row in tqdm(filtered_df.iterrows(), total=filtered_df.shape[0]):
    try:
        nearest_point = find_nearest_point(graph, row['coordinates'], darmstadt_points)
        accumulate_route_intensity(
            graph, origin_point=row['coordinates'], destination_point=nearest_point, intensity=row['Gesamt (Anzahl)']
        )
    except Exception as e:
        print(f"Error processing {row}: {e}")

# Create a custom green-to-red color map
colormap = cm.LinearColormap(
    colors=['green', 'lime', 'yellow', 'orange', 'red', 'darkred'], vmin=min(segment_intensity.values()),
    vmax=max(segment_intensity.values()), caption='Commuter Intensity'
)
map.add_child(colormap)


def add_segments_to_map(map, graph, segment_intensity):
    for segment, intensity in segment_intensity.items():
        coords = [(graph.nodes[segment[0]]['y'], graph.nodes[segment[0]]['x']),
                  (graph.nodes[segment[1]]['y'], graph.nodes[segment[1]]['x'])]
        color = colormap(intensity)
        popup = folium.Popup(f'Commuters: {intensity}', parse_html=True)
        folium.PolyLine(coords, color=color, weight=5, opacity=0.8, popup=popup).add_to(map)


add_segments_to_map(map, graph, segment_intensity)

for point in darmstadt_points:
    folium.Marker(
        location=point, icon=folium.Icon(icon='plus', prefix='fa', color='black'), popup=f'Destination: {point}'
    ).add_to(map)

map.save(output_dir / 'commuter_map_with_intensity2.html')

map_tmp = map

100%|██████████| 141/141 [04:55<00:00,  2.09s/it]


In [18]:
# Now that we have a map of the commuter intensity for of the inleading roads, no we want to place points on the map that represent Park and Ride locations
# We will use these P+R locations to with different hyperparamters such as parking space and average using itensity
# and then subtract the number of commuters on this route after some of the commuters have switched to the P+R location

map_pr = map_tmp
# Define Park and Ride locations with hyperparameters (number of parking spaces and average usage intensity)

import random


park_and_ride_locations = [
    {'coordinates': (49.92062668747579, 8.654081603730377), 'parking_spaces': 59, 'average_usage_intensity': 0.68},
    {'coordinates': (49.73991132883085, 8.613724110057003), 'parking_spaces': 52, 'average_usage_intensity': 0.88 },
    {'coordinates': (49.88784836962342, 8.631497748361191), 'parking_spaces': 480, 'average_usage_intensity': random.uniform(0.76, 0.91)},
    # Add more P+R locations as needed
]

# Function to adjust commuter intensity based on P+R usage
def adjust_commuter_intensity_with_pr(segment_intensity, pr_locations):
    for pr_location in pr_locations:
        pr_point = pr_location['coordinates']
        parking_spaces = pr_location['parking_spaces']
        usage_intensity = pr_location['average_usage_intensity']
        # occupancy rate per car during rush hour
        occupancy_rate = 1.2
        print(f"Usage intensity: {usage_intensity}, Parking spaces: {parking_spaces}, Occupancy rate: {occupancy_rate}")
        # Calculate the number of commuters switching to this P+R location
        commuters_switched = parking_spaces * usage_intensity * occupancy_rate

        # Find the nearest destination point to the P+R location
        nearest_destination = find_nearest_point(graph, pr_point, darmstadt_points)

        # Update the segment intensity by subtracting the switched commuters
        accumulate_route_intensity(graph, pr_point, nearest_destination, -commuters_switched)

# Adjust commuter intensity with the P+R locations
adjust_commuter_intensity_with_pr(segment_intensity, park_and_ride_locations)

# Update the segments on the map with the adjusted intensities
add_segments_to_map(map_pr, graph, segment_intensity)

# Add Park and Ride locations to the map
for pr_location in park_and_ride_locations:
    folium.Marker(
        location=pr_location['coordinates'],
        icon=folium.Icon(icon='parking', prefix='fa', color='blue'),
        popup=f'P+R: {pr_location["coordinates"]}, Spaces: {pr_location["parking_spaces"]}, Usage: {pr_location["average_usage_intensity"]}'
    ).add_to(map_pr)

# Save the updated map with P+R adjustments
map.save(output_dir / 'commuter_map_with_pr_adjustments.html')


Usage intensity: 0.68, Parking spaces: 59, Occupancy rate: 1.2
Usage intensity: 0.88, Parking spaces: 52, Occupancy rate: 1.2
Usage intensity: 0.8288077549382104, Parking spaces: 480, Occupancy rate: 1.2
